# Market demand prediction
*This notebook illustrates methods to predict the demand series of a supply chain.*  
*Use the virtual environment logproj.yml to run this notebook.*
***
Alessandro Tufano 2020

### Import packages

In [1]:
# %% append functions path
import sys; sys.path.insert(0, '..') #add the above level with the package

from IPython.display import display, HTML #display dataframe

#import utilities
from logproj.utilities import creaCartella

### Set data fields

In [2]:
string_casestudy = 'TOY_DATA'

### Import data

In [3]:
# %% import data
from logproj.data_generator_distribution import generateDistributionData

#random generation of distribution data
_, _, _, D_mov = generateDistributionData(num_movements=500)


In [4]:
#print nodes dataframe
display(HTML(D_mov.to_html()))

,LOADING_NODE,LOADING_NODE_DESCRIPTION,LOADING_NODE_LATITUDE,LOADING_NODE_LONGITUDE,PTA_FROM,PTD_FROM,ATA_FROM,ATD_FROM,DISCHARGING_NODE,DISCHARGING_NODE_DESCRIPTION,DISCHARGING_LATITUDE,DISCHARGING_LONGITUDE,PTA_TO,PTD_TO,ATA_TO,ATD_TO,ITEMCODE,PRODUCT_FAMILY,CLIENT,VEHICLE_CODE,VOYAGE_CODE,QUANTITY,TIMESTAMP_IN,PACKAGE_DESCRIPTION,USER
0,12,NODE_12,41.486247,14.731834,2020-01-02 00:00:00.000000,2020-01-02 01:00:00.000000,2020-01-01 23:31:17.640671,2020-01-02 01:21:33.129862,5,NODE_5,41.701536,14.423318,2020-01-02 10:31:01.114363,2020-01-02 11:31:01.114363,2020-01-02 10:21:55.037935,2020-01-02 11:42:39.193208,0,PRODUCT_FAMILY 1,CLIENT 2,TRUCK 1,0,5.024001,2019-12-30 10:09:18.212913,TEU CONTAINER,USER_3
0,19,NODE_19,41.882769,14.427607,2020-01-02 11:47:51.369417,2020-01-02 12:47:51.369417,2020-01-02 11:22:47.712760,2020-01-02 12:42:13.635916,12,NODE_12,41.486247,14.731834,2020-01-03 02:39:59.188300,2020-01-03 03:39:59.188300,2020-01-03 02:57:13.282048,2020-01-03 03:45:31.046350,0,PRODUCT_FAMILY 1,CLIENT 2,TRUCK 1,0,5.362749,2019-12-16 13:20:43.999587,TEU CONTAINER,USER_4
0,8,NODE_8,41.544122,14.386166,2020-01-03 04:45:31.577913,2020-01-03 05:45:31.577913,2020-01-03 04:25:51.884309,2020-01-03 06:03:54.506110,2,NODE_2,41.724386,14.543246,2020-01-03 13:06:16.086164,2020-01-03 14:06:16.086164,2020-01-03 12:54:43.141959,2020-01-03 14:06:14.228795,1,PRODUCT_FAMILY 2,CLIENT 2,TRUCK 1,0,5.379027,2019-12-24 10:30:24.107775,TEU CONTAINER,USER_4
0,15,NODE_15,41.624813,14.829680,2020-01-03 16:00:34.782137,2020-01-03 17:00:34.782137,2020-01-03 16:01:27.104886,2020-01-03 16:51:36.948851,6,NODE_6,41.852532,14.236997,2020-01-04 06:38:46.493746,2020-01-04 07:38:46.493746,2020-01-04 06:31:52.928520,2020-01-04 07:34:01.555398,0,PRODUCT_FAMILY 1,CLIENT 2,TRUCK 1,0,9.652351,2020-01-01 00:02:17.815728,TEU CONTAINER,USER_0
0,10,NODE_10,41.541457,14.956873,2020-01-04 07:41:35.339301,2020-01-04 08:41:35.339301,2020-01-04 07:43:59.094970,2020-01-04 08:49:44.880386,8,NODE_8,41.544122,14.386166,2020-01-04 18:45:44.638291,2020-01-04 19:45:44.638291,2020-01-04 18:36:13.026864,2020-01-04 19:43:45.240435,0,PRODUCT_FAMILY 1,CLIENT 2,TRUCK 1,0,3.776112,2020-01-02 02:15:17.967232,TEU CONTAINER,USER_7
0,23,NODE_23,41.944142,13.998359,2020-01-04 22:03:08.424643,2020-01-04 23:03:08.424643,2020-01-04 21:46:33.332771,2020-01-04 23:13:25.885334,21,NODE_21,41.656260,14.682803,2020-01-06 03:14:43.222258,2020-01-06 04:14:43.222258,2020-01-06 03:12:17.516020,2020-01-06 04:15:05.725170,0,PRODUCT_FAMILY 1,CLIENT 2,TRUCK 1,0,3.063135,2019-12-27 03:56:18.560034,FEU CONTAINER,USER_0
0,24,NODE_24,41.551214,14.383735,2020-01-06 04:55:59.950535,2020-01-06 05:55:59.950535,2020-01-06 05:02:58.488313,2020-01-06 05:59:58.703032,3,NODE_3,41.649771,14.336380,2020-01-06 10:20:47.283201,2020-01-06 11:20:47.283201,2020-01-06 10:11:06.997086,2020-01-06 11:29:10.001311,0,PRODUCT_FAMILY 1,CLIENT 2,TRUCK 1,0,6.997336,2020-01-02 03:54:11.156294,FEU CONTAINER,USER_6
0,3,NODE_3,41.649771,14.336380,2020-01-06 11:35:21.622534,2020-01-06 12:35:21.622534,2020-01-06 11:38:37.110794,2020-01-06 12:41:33.750831,20,NODE_20,41.875913,14.552090,2020-01-06 20:33:02.906236,2020-01-06 21:33:02.906236,2020-01-06 20:30:40.007869,2020-01-06 21:24:36.520013,1,PRODUCT_FAMILY 2,CLIENT 1,TRUCK 1,0,6.556406,2019-12-24 02:37:55.886979,TEU CONTAINER,USER_3
0,22,NODE_22,41.525834,14.302754,2020-01-06 23:33:18.477763,2020-01-07 00:33:18.477763,2020-01-06 23:50:16.475198,2020-01-07 00:30:44.224366,15,NODE_15,41.624813,14.829680,2020-01-07 17:19:41.995397,2020-01-07 18:19:41.995397,2020-01-07 18:01:01.484770,2020-01-07 17:38:30.444761,0,PRODUCT_FAMILY 1,CLIENT 1,TRUCK 1,0,1.613994,2020-01-03 05:37:31.846185,TEU CONTAINER,USER_6
0,3,NODE_3,41.649771,14.336380,2020-01-07 18:27:52.215747,2020-01-07 19:27:52.215747,2020-01-07 18:35:19.366080,2020-01-07 19:52:02.622467,7,NODE_7,41.516428,14.357767,2020-01-08 00:16:01.415146,2020-01-08 01:16:01.415146,2020-01-08 00:21:47.616146,2020-01-08 01:21:27.829444,0,PRODUCT_FAMILY 1,CLIE

### Create folder hierarchy

In [5]:
# %% create folder hierarchy
pathResults = 'C:\\Users\\aletu\\desktop'
_, root_path = creaCartella(pathResults,f"{string_casestudy}_results")
_, prediction_results_path = creaCartella(root_path,f"P9_predictions")

Cartella TOY_DATA_results già esistente
Cartella P9_predictions già esistente


### Build the time series

In [6]:
D_time = D_mov.groupby(['TIMESTAMP_IN']).agg({'QUANTITY':['sum'],'TIMESTAMP_IN':['size']}).reset_index()
D_time.columns = ['TIMESTAMP', 'QUANTITY','COUNT']
#print the series
display(HTML(D_time.to_html()))

,TIMESTAMP,QUANTITY,COUNT
0,2019-12-16 13:20:43.999587,5.362749,1
1,2019-12-23 18:22:07.011613,6.620328,1
2,2019-12-24 02:37:55.886979,6.556406,1
3,2019-12-24 10:30:24.107775,5.379027,1
4,2019-12-24 22:38:11.369152,4.095256,1
5,2019-12-27 03:56:18.560034,3.063135,1
6,2019-12-28 14:19:33.078793,6.982206,1
7,2019-12-30 07:57:20.352631,5.836694,1
8,2019-12-30 10:09:18.212913,5.024001,1
9,2019-12-30 12:29:21.711395,2.791345,1


### SARIMA predictions

In [7]:
from logproj.P9_workloadPrediction.demand_predicter import LOOP_PREDICT_SARIMA
LOOP_PREDICT_SARIMA(D_time, 
                    date_field='TIMESTAMP',
                    qtyVariable='QUANTITY',
                    countVariable='COUNT',
                    prediction_results_path = prediction_results_path, 
                    filterVariable = [],
                    samplingInterval='week')
#check the destination folder to analyse the results

ImportError: cannot import name 'plot_plotly'

### FBPROPHET predictions

In [ ]:
from logproj.P9_workloadPrediction.demand_predicter import LOOP_PREDICT_FBPROPHET
LOOP_PREDICT_FBPROPHET(D_time, 
                       timeVariable='TIMESTAMP', 
                       qtyVariable='QUANTITY', 
                       countVariable='COUNT', 
                       prediction_results_path=prediction_results_path, 
                       filterVariable=[], 
                       samplingInterval='week')

### Predict the inventory of a vehicle

In [ ]:
# generate arc flow data using the VEHICLE STATISTICS
from logproj.P8_performanceAssessment.vehicle_assessment import D_voyageStatistics

#columns movements data
movementColumns= {
           'timevariable':'TIMESTAMP_IN', #colonna obbligatoria con i timestamp per ogni movimento
           'capacityField':'QUANTITY', #colonna obblicatoria che identifica la quantita' movimentata
           
            'vehiclefield':'VEHICLE_CODE', #colonna che identifica il veicolo utilizzato
           
            'terminalfieldFrom':'LOADING_NODE', #colonna che identifica il nodo di carico
            'terminalfieldto':'DISCHARGING_NODE', #colonna che identifica il nodo di scarico
    
            
           }

timeColumns = {'loadingpta':'PTA_FROM',
               'loadingptd':'PTD_FROM', 
               'dischargingpta':'PTA_TO', 
               'dischargingptd':'PTD_TO', 
               'loadingata':'ATA_FROM', 
               'loadingatd':'ATD_FROM', 
               'dischargingata':'ATA_TO', 
               'dischargingatd':'ATD_TO'
               }
         
        
#define arc table
_,  D_arcs_route, _  = D_voyageStatistics(D_mov,
                                    timefield=movementColumns['timevariable'],
                                    locfrom = movementColumns['terminalfieldFrom'],
                                    locto= movementColumns['terminalfieldto'],
                                    timeColumns = timeColumns,
                                    capacityField=movementColumns['capacityField'],
                                    voyagefield =movementColumns['vehiclefield'],
                                    censoredData=False, #use complete data   
                                    actual='ACTUAL') #use actual data 
#print resulting dataframe
D_arcs_route.head()

In [ ]:
#resample the time series for each vehicle
import pandas as pd
import matplotlib.pyplot as plt
D=pd.DataFrame()
for vehicle in set(D_arcs_route['VEHICLE_CODE_from']):
    #vehicle ='Vessel 1'
    D_filtered = D_arcs_route[D_arcs_route['VEHICLE_CODE_from']==vehicle]
    D_filtered['TIMESTAMP'] = D_filtered['departureFromALAP']
    D_filtered = D_filtered.sort_values(by = ['TIMESTAMP'])
    
    
    #build a equispaced series
    D_series = D_filtered[['TIMESTAMP','inventory']]
    D_series = D_series.set_index('TIMESTAMP')
    
    D_series_resampled = D_series.resample('H').mean()
    D_series_resampled['inventory'] = D_series_resampled['inventory'].fillna(method ='ffill') 
    
    #sample plot
    plt.figure()
    plt.plot(D_series_resampled.index,D_series_resampled['inventory'])
    plt.plot(D_series.index,D_series['inventory'])
    plt.title(f"Inventory resampled, vehicle: {vehicle}")
    
    #hour shift
    D_series_resampled['hour_shift'] = D_series_resampled['inventory'].shift(1)
    
    #day shift
    D_series_resampled['day_shift'] = D_series_resampled['inventory'].shift(24)
    
    #week shift
    D_series_resampled['week_shift'] = D_series_resampled['inventory'].shift(24*7)
    
    #month shift
    D_series_resampled['month_shift'] = D_series_resampled['inventory'].shift(24*30)
    D_series_resampled=D_series_resampled.dropna()
    D_series_resampled['TIMESTAMP_HOUR'] = D_series_resampled.index
    D_series_resampled['TIMESTAMP_HOUR'] = D_series_resampled['TIMESTAMP_HOUR'].dt.strftime('%Y-%m-%d %H:%M:00')
    
    #join with the original
    D_filtered['TIMESTAMP_HOUR'] = D_filtered['TIMESTAMP'].dt.strftime('%Y-%m-%d %H:%M:00')
    D_filtered = D_filtered.merge(D_series_resampled, on = 'TIMESTAMP_HOUR', how = 'left')
    D_filtered = D_filtered.dropna(subset = ['month_shift'])
    D_filtered = D_filtered.drop(columns=['TIMESTAMP','TIMESTAMP_HOUR','inventory_y'])
    D=D.append(D_filtered)

#print resulting dataframe
display(HTML(D.to_html()))

In [ ]:
#Preprocess for machine learning
#define the learning table
X=pd.DataFrame()

#define the target variable
X['y'] = D['inventory_x']


In [ ]:
def returnDummyColumnsFromList(D, columnName):
    '''
    extract the value from a pandas cell where a list is saved

    Parameters
    ----------
    D : TYPE pandas dataframe
        DESCRIPTION.
    columnName : TYPE
        DESCRIPTION.

    Returns
    -------
    s : TYPE column string name
        DESCRIPTION.

    '''
    x = D[columnName].values
    #print(x)
    #print(type(x))
    newList=[]
    for j in x:
        for i in j:
            i = str(i)
            yyy = i.replace('nan,','')
            yyy = yyy.replace('nan','')
            newList.append(ast.literal_eval(yyy))


    s = pd.Series(newList)
    #X_add=pd.get_dummies(s.apply(pd.Series).stack()).sum(level=0)
    #print(s)
    return s

In [ ]:
#convert variables extracting from the list
from logproj.ml_dataCleaning import transformClockData, dummyColumns


month_cos, month_sin=transformClockData(D['departureFromALAP'].dt.month)
day_cos, day_sin=transformClockData(D['departureFromALAP'].dt.day)
hour_cos, hour_sin=transformClockData(D['departureFromALAP'].dt.hour)
min_cos, min_sin=transformClockData(D['departureFromALAP'].dt.hour)

X['DepYear']=D['departureFromALAP'].dt.year
X['DepMonthC']=month_cos
X['DepMonthS']=month_sin
X['DepDayC']=day_cos
X['DepDayS']=day_sin
X['DepHourC']=hour_cos
X['DepHourS']=hour_sin
X['DepMinuteC']=min_cos
X['DepMinuteS']=min_sin

#ADD VARIABLE DEPARTURE FROM ASAP
month_cos, month_sin=transformClockData(D['arrivalToASAP'].dt.month)
day_cos, day_sin=transformClockData(D['arrivalToASAP'].dt.day)
hour_cos, hour_sin=transformClockData(D['arrivalToASAP'].dt.hour)
min_cos, min_sin=transformClockData(D['arrivalToASAP'].dt.hour)

X['ArrYear']=D['arrivalToASAP'].dt.year
X['ArrMonthC']=month_cos
X['ArrMonthS']=month_sin
X['ArrDayC']=day_cos
X['ArrDayS']=day_sin
X['ArrHourC']=hour_cos
X['ArrHourS']=hour_sin
X['ArrMinuteC']=min_cos
X['ArrMinuteS']=min_sin


# ADD VEHICLE CODE
X['Vessel']=D['VEHICLE_CODE_from']

# ADD LOCATIONS
X['LocationFrom']=D['arcFrom']
X['LocationTo']=D['arcTo']


# CONVERT LOADING_NODE_LATITUDE_from
X_add = returnDummyColumnsFromList(D, 'LOADING_NODE_LATITUDE_from')
X['LOADING_NODE_LATITUDE_from'] = list(X_add)


# CONVERT LOADING_NODE_LONGITUDE_from
X_add = returnDummyColumnsFromList(D, 'LOADING_NODE_LONGITUDE_from')
X['LOADING_NODE_LONGITUDE_from'] = list(X_add)



# CONVERT DISCHARGING_LATITUDE_from
X_add = returnDummyColumnsFromList(D, 'DISCHARGING_LATITUDE_from')
X['DISCHARGING_LATITUDE_from'] = list(X_add)


# CONVERT DISCHARGING_LONGITUDE_from
X_add = returnDummyColumnsFromList(D, 'DISCHARGING_LONGITUDE_from')
X['DISCHARGING_LONGITUDE_from'] = list(X_add)


# CONVERT ITEMCODE_from
X_add = returnDummyColumnsFromList(D, 'ITEMCODE_from')
X['ITEMCODE_from'] = list(X_add)


# CONVERT STRINGS TO DUMMIES
X = dummyColumns(X)
X.head()

In [ ]:
# %% Extract X, and y
X=X.dropna()
y=X['y']
X=X.drop(columns=['y'])
X

In [ ]:
# %% build model
from logproj.M_learningMethod import train_model

# import models
# add linear models
from logproj.M_learningMethod.linear_models import models_regression as lin_class_mod

#add bayesian models
#from logproj.M_learningMethod.bayesians_models import models_regression as bay_class_mod

#add symbolist models
from logproj.M_learningMethod.symbolists_models import models_regression as sym_class_mod

#add connectionist models
from logproj.M_learningMethod.connectionists_models import models_regression as con_class_mod

#add analogizers models
from logproj.M_learningMethod.analogizers_models import models_regression as ana_class_mod

#add ensemble methods
from logproj.M_learningMethod.ensemble_methods import models_regression as ens_class_mod

all_models_regression={}
all_models_regression.update(lin_class_mod)
#all_models_regression.update(bay_class_mod)
all_models_regression.update(sym_class_mod)
all_models_regression.update(con_class_mod)
all_models_regression.update(ana_class_mod)
all_models_regression.update(ens_class_mod)


D_res_regr = train_model.train_models_regression(X,y,all_models_regression)

In [ ]:
#print results of the performance of the models
display(HTML(D_res_regr.to_html()))